## <font color = "brown"> Importing Libraries </font>

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## <font color = "brown"> Read Data </font>

In [15]:
# Function to get and transform data
def data():
    # Read data
    df = pd.read_csv("../../raw_data/market_data.csv").reset_index(drop = True)
    df["Time (UTC+10)"] = pd.to_datetime(df["Time (UTC+10)"]) # Convert data type

    # Set data
    vicp = df["Regions VIC Trading Price ($/MWh)"]
    vicd = df["Regions VIC Operational Demand (MW)"]
    time = df["Time (UTC+10)"]

    # Creating DataFrame for Prices
    price = pd.DataFrame()
    price["Time"] = time
    price["Price"] = vicp

    # Creating DataFrame for Demands
    demand = pd.DataFrame()
    demand["Time"] = time
    demand["Demand"] = vicd
    
    return df, price, demand

## <font color = "brown"> Moving Averages </font>

In [16]:
# Function to calculate the 2n-centered moving average
def CMA(df, col, n):
    MA = []
    dfc = df.reset_index(drop = True)
    
    for i in dfc.index:
        if (i > n - 1) and i < len(df) - n:
            Period = []
            for j in range(1, n + 1):
                Period.append(df[col].loc[i - j])
                Period.append(df[col].loc[i + j])
            entry = np.mean(Period)
        else:
            entry = np.nan

        MA.append(entry)
    
    dfn = pd.DataFrame()
    dfn["MA"] = MA
    
    dff = pd.merge(dfc, dfn, left_index = True, right_index = True, how = "inner")
    return dff.dropna().reset_index(drop = True)

In [17]:
# Function to intersect MA position between price and demand
def intersect(price, demand, status):
    inter = pd.merge(price, demand, on = "Time", how = "inner")
    inter["Status"] = status
    return inter.drop(["MA_x", "MA_y"], axis = 1)

In [18]:
# Function to assign action to each time point
def BiggerPicture(df, Charge, Discharge):
    cdc = pd.concat([Charge, Discharge], ignore_index = True)

    cdc = cdc[["Time", "Status"]]
    dff = pd.merge(df, cdc, left_on = "Time (UTC+10)", right_on = "Time", how = "left")
    dff = dff[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)", "Status"]]
    dff.columns = ["Time", "Price", "Status"]
    
    return dff.replace(np.nan, "Do Nothing")

In [19]:
# Function to generate output
def Moving(df, price, demand, n):
    # Create Moving Average Price Data
    PriceMA = CMA(price, "Price", n)
    PriceMA.columns = ["Time", "Price", "MA"]
    
    # Divide Price Data
    UpperPrice = PriceMA[PriceMA.Price >= PriceMA.MA]
    LowerPrice = PriceMA[PriceMA.Price < PriceMA.MA]
    
    # Create Moving Average Demand Data
    DemandMA = CMA(demand, "Demand", n)
    DemandMA.columns = ["Time", "Demand", "MA"]
    
    # Divide Price Data
    UpperDemand = DemandMA[DemandMA.Demand >= DemandMA.MA]
    LowerDemand = DemandMA[DemandMA.Demand < DemandMA.MA]
    
    # Determine Charge-Discharge Time
    Charge = intersect(LowerPrice, LowerDemand, "Charge")
    Discharge = intersect(UpperPrice, UpperDemand, "Discharge")
    
    # Assign Action to Time Point
    final = BiggerPicture(df, Charge, Discharge)
    
    return final

In [20]:
def visualise(df):
    curdf = df["Status"].replace(["Charge", "Discharge", "Do Nothing"], [1, 3, 2])
    sns.lineplot(y = curdf, x = curdf.index)

In [21]:
def optimise(MA = 10):
    df, price, demand = data()
    final = Moving(df, price, demand, MA)
    #visualise(final)
    return final

In [37]:
optimise_df = optimise(10)
optimise_df

,Time,Price,Status
0,2018-01-01 00:00:00,90.43,Do Nothing
1,2018-01-01 00:30:00,92.46,Do Nothing
2,2018-01-01 01:00:00,87.62,Do Nothing
3,2018-01-01 01:30:00,73.08,Do Nothing
4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...
63452,2021-08-14 22:00:00,49.93,Do Nothing
63453,2021-08-14 22:30:00,62.86,Do Nothing
63454,2021-08-14 23:00:00,32.26,Do Nothing
63455,2021-08-14 23:30:00,25.10,Do Nothing


In [ ]:
optimise_df.to_csv('moving_average_model.csv')

## <font color = "brown"> Calculate Revenue </font>

In [60]:
# define variables
BATTERY_CAPACITY = 580 # maximum capacity MWH
DISCHARGE_EFFICIENCY = 0.9
CHARGE_EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
MAX_CAPACITY = 300

In [61]:
# generate_capacity calculates all the columns needed to calculate revenue
# and takes dataframe with columns ['Time', 'Price', 'Status'] as input

def generate_capacity(df):
    
    # initialises variable
    raw_power = 0
    opening_capacity = 0
    closing_capacity = 0
    market_dispatch = 0

    raw_power_s = pd.Series(dtype = float)
    opening_capacity_s = pd.Series(dtype = float)
    closing_capacity_s = pd.Series(dtype = float)
    market_dispatch_s = pd.Series(dtype = float)
    
    status = df['Status']
    
    for i in df.index:
        if i == 0: # set opening capacity
            opening_capacity_s.loc[i] = 0
        else: # set opening capacity to previous closing capacity
            opening_capacity = closing_capacity_s.loc[i-1]
            opening_capacity_s.loc[i] = closing_capacity_s.loc[i-1]
        if status[i] == 'Charge':
            raw_power = -min(MAX_CAPACITY, ((int(BATTERY_CAPACITY) - float(opening_capacity_s[i]))/CHARGE_EFFICIENCY) * 2) # 300 is the minimum capacity
            market_dispatch = raw_power / 2
            closing_capacity = max(0, min(opening_capacity_s[i] - market_dispatch * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
            raw_power_s.loc[i] = raw_power
            market_dispatch_s.loc[i] = market_dispatch
            closing_capacity_s.loc[i] = closing_capacity
        elif status[i] == 'Discharge':
            raw_power = min(MAX_CAPACITY, (float(opening_capacity_s[i]) / DISCHARGE_EFFICIENCY) * 2)
            market_dispatch = raw_power/2 * DISCHARGE_EFFICIENCY
            closing_capacity = max(0, min(opening_capacity_s[i] - (market_dispatch * 1/DISCHARGE_EFFICIENCY), BATTERY_CAPACITY))
            market_dispatch_s.loc[i] = market_dispatch
            closing_capacity_s.loc[i] = closing_capacity
            raw_power_s.loc[i] = raw_power
        else:
            raw_power_s.loc[i] = 0
            market_dispatch_s.loc[i] = 0
            closing_capacity_s.loc[i] = closing_capacity
    
    df['raw power'] = raw_power_s
    df['market dispatch'] = market_dispatch_s
    df['opening capacity'] = opening_capacity_s
    df['closing capacity'] = closing_capacity_s

In [62]:
# calculate_revenue calculates revenue from 2018-2021

def calculate_revenue(df):
    revenue_s = pd.Series(dtype = float)
    for i in df.index:
        if df['market dispatch'].loc[i] < 0:
            revenue_s.loc[i] = df['Price'].loc[i] * df['market dispatch'].loc[i] * (1 / MARGINAL_LOSS_FACTOR)
        else:
            revenue_s.loc[i] = df['Price'].loc[i] * df['market dispatch'].loc[i] * (MARGINAL_LOSS_FACTOR)
            
    df['revenue'] = revenue_s
    return sum(df['revenue'])

In [63]:
generate_capacity(optimise_df)

In [64]:
calculate_revenue(optimise_df)

94407856.08089435

In [65]:
optimise_df

,Time,Price,Status,raw power,market dispatch,opening capacity,closing capacity,revenue
0,2018-01-01 00:00:00,90.43,Do Nothing,0.0,0.0,0.0,0.0,0.0
1,2018-01-01 00:30:00,92.46,Do Nothing,0.0,0.0,0.0,0.0,0.0
2,2018-01-01 01:00:00,87.62,Do Nothing,0.0,0.0,0.0,0.0,0.0
3,2018-01-01 01:30:00,73.08,Do Nothing,0.0,0.0,0.0,0.0,0.0
4,2018-01-01 02:00:00,70.18,Do Nothing,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
63452,2021-08-14 22:00:00,49.93,Do Nothing,0.0,0.0,0.0,0.0,0.0
63453,2021-08-14 22:30:00,62.86,Do Nothing,0.0,0.0,0.0,0.0,0.0
63454,2021-08-14 23:00:00,32.26,Do Nothing,0.0,0.0,0.0,0.0,0.0
63455,2021-08-14 23:30:00,25.10,Do Nothing,0.0,0.0,0.0,0.0,0.0
